In [1]:
!pip install git+https://github.com/shuiruge/neural-ode.git@master

  Cloning https://github.com/shuiruge/neural-ode.git (to revision master) to /tmp/pip-req-build-x_5eyx0p
  Running command git clone -q https://github.com/shuiruge/neural-ode.git /tmp/pip-req-build-x_5eyx0p
  Created wheel for node: filename=node-0.1.0-cp36-none-any.whl size=37076 sha256=f5cee0fd290271ed7027484d8bf8e8f7fa7a2a7534706006f69ee73d4fbccdff
  Stored in directory: /tmp/pip-ephem-wheel-cache-6trp16vl/wheels/36/41/e1/1cf7fd120543ff07c299bee3a2ce3fe659795c54f7e03fe9b6
Successfully built node


In [2]:
import numpy as np
import tensorflow as tf
from node.hopfield import ContinuousTimeHopfieldLayer
                           
# for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

tf.keras.backend.clear_session()


IMAGE_SIZE = (28, 28)


def pooling(x, size):
  # x shape: [None, width, height]
  x = tf.expand_dims(x, axis=-1)
  x = tf.image.resize(x, size)
  return x  # shape: [None, size[0], size[1], 1]


def process_data(X, y, image_size):
  X = pooling(X, image_size)
  X = X / 255.
  X = tf.where(X < 0.5, -1., 1.)
  X = tf.reshape(X, [-1, image_size[0] * image_size[1]])
  y = tf.one_hot(y, 10)
  return tf.cast(X, tf.float32), tf.cast(y, tf.float32)


def get_benchmark_model(model):
  layers = [
    layer for layer in model.layers
    if not isinstance(layer, ContinuousTimeHopfieldLayer)]
  return tf.keras.Sequential(layers)


model = tf.keras.Sequential([
  tf.keras.Input([IMAGE_SIZE[0] * IMAGE_SIZE[1]]),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dense(512, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(256, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(128, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax'),
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

mnist = tf.keras.datasets.mnist
(x_train, y_train), _ = mnist.load_data()
x_train, y_train = process_data(x_train, y_train, IMAGE_SIZE)
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(1000).repeat(50).batch(128)
model.fit(dataset)

23438/23438 [==============================] - 195s 8ms/step - loss: 0.0528 - acc: 0.9922


Exame the noise effect:

In [3]:
FLIP_RATIO = 0.3

benchmark_model = get_benchmark_model(model)

X = x_train[-2000:]
targets = np.argmax(y_train[-2000:], axis=1)
noised_X = np.where(np.random.random(size=X.shape) < FLIP_RATIO,
                    -X, X)
unoised_y = np.argmax(model.predict(X), axis=1)
y = np.argmax(model.predict(noised_X), axis=1)
benchmark_y = np.argmax(benchmark_model.predict(noised_X), axis=1)

num_misleading = 0
num_corrected = 0
num_uncorrected = 0
for i, (benchmark_yi, yi, unoised_yi, ti) in enumerate(zip(
    benchmark_y, y, unoised_y, targets)):
  if yi == unoised_yi and benchmark_yi != yi:
    num_corrected += 1
  elif benchmark_yi == unoised_yi and benchmark_yi != yi:
    num_misleading += 1
  elif yi != unoised_yi and benchmark_yi == yi:
    num_uncorrected += 1
  else:
    pass
  if i < 50:
    print(f'{benchmark_yi} => {yi} | {unoised_yi} ({ti})')
print(
  f'misleading ratio: {num_misleading / X.shape[0]}',
  f'corrected ratio: {num_corrected / X.shape[0]}',
  f'uncorrected ratio: {num_uncorrected / X.shape[0]}')

3 => 1 | 1 (2)
7 => 1 | 1 (3)
0 => 5 | 1 (0)
5 => 5 | 1 (4)
5 => 5 | 1 (9)
7 => 1 | 5 (5)
7 => 1 | 5 (9)
5 => 5 | 5 (6)
1 => 1 | 1 (1)
1 => 1 | 1 (7)
5 => 5 | 5 (6)
7 => 5 | 1 (8)
7 => 5 | 5 (0)
5 => 5 | 5 (9)
7 => 1 | 1 (0)
1 => 1 | 1 (0)
8 => 1 | 5 (6)
7 => 1 | 1 (1)
4 => 1 | 1 (4)
1 => 1 | 1 (2)
5 => 5 | 1 (8)
5 => 5 | 1 (3)
1 => 1 | 1 (1)
7 => 1 | 1 (4)
5 => 5 | 1 (8)
5 => 5 | 5 (5)
2 => 1 | 1 (2)
7 => 1 | 5 (6)
1 => 1 | 1 (8)
7 => 1 | 1 (7)
1 => 1 | 1 (2)
5 => 5 | 1 (8)
2 => 1 | 1 (2)
3 => 5 | 1 (1)
5 => 5 | 5 (9)
7 => 1 | 1 (0)
5 => 1 | 1 (0)
7 => 5 | 1 (4)
5 => 5 | 1 (0)
5 => 1 | 5 (5)
1 => 5 | 5 (5)
5 => 1 | 5 (6)
8 => 1 | 1 (7)
1 => 1 | 5 (6)
5 => 5 | 1 (2)
5 => 5 | 1 (3)
0 => 1 | 5 (8)
5 => 5 | 1 (4)
7 => 1 | 1 (3)
7 => 5 | 5 (4)
misleading ratio: 0.0435 corrected ratio: 0.4305 uncorrected ratio: 0.1365


In [4]:
for layer in model.layers:
  if isinstance(layer, ContinuousTimeHopfieldLayer):
    print(layer._stop_condition.relax_time.numpy())

267.9454
227.85867
282.02838
